In [1]:
# Cellule 1 : Télécharger le ZIP si pas déjà fait
import zipfile
from pathlib import Path
import httpx

AN_ZIP_URL = "http://data.assemblee-national.fr/static/openData/repository/17/loi/scrutins/Scrutins.xml.zip"
cache_dir = Path(".cache/an")
zip_path = cache_dir / "Scrutins.xml.zip"

if not zip_path.exists():
    print("📥 Téléchargement du ZIP...")
    cache_dir.mkdir(parents=True, exist_ok=True)
    with httpx.stream("GET", AN_ZIP_URL, timeout=120.0, follow_redirects=True) as r:
        r.raise_for_status()
        with open(zip_path, "wb") as f:
            for chunk in r.iter_bytes():
                f.write(chunk)
    print("✅ Téléchargé")
else:
    print("✅ ZIP déjà en cache")

# Cellule 2 : Lister le contenu du ZIP
with zipfile.ZipFile(zip_path, "r") as zf:
    xml_files = [n for n in zf.namelist() if n.lower().endswith(".xml")]
    print(f"📁 Fichiers XML trouvés dans le ZIP: {len(xml_files)}")
    for name in xml_files[:5]:  # affiche les 5 premiers
        print(f"  - {name}")

# Cellule 3 : Test si acteurRef est présent
with zipfile.ZipFile(zip_path, "r") as zf:
    xml_name = xml_files[0]  # prend le premier XML
    data = zf.read(xml_name).decode('utf-8', errors='ignore')
    
    acteur_count = data.count('acteurRef')
    print(f"\n🔍 Occurrences de 'acteurRef': {acteur_count}")
    
    if acteur_count > 0:
        print("✅ Le XML contient des acteurRef (votes nominatifs présents)")
        # Afficher un extrait
        idx = data.find('acteurRef')
        if idx > 0:
            print("\n📄 Extrait autour du premier acteurRef:")
            print(data[max(0, idx-200):idx+200])
    else:
        print("❌ Aucun acteurRef trouvé → ce ZIP ne contient PAS les votes nominatifs")

# Cellule 4 : Parser un scrutin pour voir la structure
from lxml import etree

with zipfile.ZipFile(zip_path, "r") as zf:
    with zf.open(xml_files[0]) as f:
        tree = etree.parse(f)
        root = tree.getroot()
        
        # Chercher un scrutin
        scrutins = root.xpath(".//*[local-name()='scrutin']")
        print(f"\n📊 Nombre de scrutins trouvés: {len(scrutins)}")
        
        if scrutins:
            scrutin = scrutins[0]
            print("\n🔬 Structure du premier scrutin:")
            print(etree.tostring(scrutin, encoding='unicode', pretty_print=True)[:2000])
            
            # Test extraction acteurRef
            actors = scrutin.xpath(".//*[local-name()='acteurRef']")
            print(f"\n👥 acteurRef dans ce scrutin: {len(actors)}")
            if actors:
                print(f"   Exemple: {actors[0].text}")

✅ ZIP déjà en cache
📁 Fichiers XML trouvés dans le ZIP: 5294
  - xml/VTANR5L17V3252.xml
  - xml/VTANR5L17V1392.xml
  - xml/VTANR5L17V3913.xml
  - xml/VTANR5L17V3096.xml
  - xml/VTANR5L17V836.xml

🔍 Occurrences de 'acteurRef': 584
✅ Le XML contient des acteurRef (votes nominatifs présents)

📄 Extrait autour du premier acteurRef:
nVotantsVolontaires>
            </decompteVoix>
            <decompteNominatif>
              <nonVotants/>
              <pours/>
              <contres>
                <votant>
                  <acteurRef>PA793432</acteurRef>
                  <mandatRef>PM840276</mandatRef>
                  <parDelegation>false</parDelegation>
                  <numPlace>080</numPlace>
                </vot

📊 Nombre de scrutins trouvés: 0


In [3]:
# Cellule 1 : Télécharger le ZIP si pas déjà fait
import zipfile
from pathlib import Path
import httpx

AN_ZIP_URL = "http://data.assemblee-nationale.fr/static/openData/repository/17/loi/scrutins/Scrutins.xml.zip"
cache_dir = Path(".cache/an")
zip_path = cache_dir / "Scrutins.xml.zip"

if not zip_path.exists():
    print("📥 Téléchargement du ZIP...")
    cache_dir.mkdir(parents=True, exist_ok=True)
    with httpx.stream("GET", AN_ZIP_URL, timeout=120.0, follow_redirects=True) as r:
        r.raise_for_status()
        with open(zip_path, "wb") as f:
            for chunk in r.iter_bytes():
                f.write(chunk)
    print("✅ Téléchargé")
else:
    print("✅ ZIP déjà en cache")

# Cellule 2 : Lister le contenu du ZIP
with zipfile.ZipFile(zip_path, "r") as zf:
    xml_files = [n for n in zf.namelist() if n.lower().endswith(".xml")]
    print(f"📁 Fichiers XML trouvés dans le ZIP: {len(xml_files)}")
    for name in xml_files[:5]:
        print(f"  - {name}")

# Cellule 3 : Définir la fonction _extract_votes
def _extract_votes(scrutin_node) -> list[dict]:
    """
    Extraction robuste et namespace-agnostic :
    - prend tous les acteurRef du scrutin
    - déduit la position en remontant les parents (pour/contre/abstention/nonVotant)
    - récupère organeRef (groupe) si présent dans les parents
    """
    votes = []

    bucket_to_pos = {
        "pour": "FOR",
        "pours": "FOR",
        "contre": "AGAINST",
        "contres": "AGAINST",
        "abstention": "ABSTAIN",
        "abstentions": "ABSTAIN",
        "nonVotant": "NONVOTING",
        "nonvotant": "NONVOTING",
        "nonVotants": "NONVOTING",
    }

    actor_nodes = scrutin_node.xpath(".//*[local-name()='acteurRef']")
    print(f"🔍 Nombre d'acteurRef trouvés: {len(actor_nodes)}")
    
    for a in actor_nodes:
        pid = (a.text or "").strip()
        if not pid:
            continue

        pos = None
        group = None
        cur = a
        
        for _ in range(0, 30):
            cur = cur.getparent()
            if cur is None:
                break

            lname = cur.tag.split("}")[-1] if isinstance(cur.tag, str) else ""
            if pos is None and lname in bucket_to_pos:
                pos = bucket_to_pos[lname]

            if group is None:
                g = cur.xpath("string(.//*[local-name()='organeRef'][1])")
                g = g.strip() if isinstance(g, str) else ""
                if g:
                    group = g

            if pos is not None and group is not None:
                break

        if pos is None:
            print(f"⚠️  Position non trouvée pour {pid}")
            continue

        votes.append({
            "person_id": pid,
            "position": pos,
            "group": group,
            "constituency": None,
            "name": None,
        })

    print(f"✅ Votes extraits: {len(votes)}")
    
    # dédoublonnage
    seen = set()
    uniq = []
    for v in votes:
        key = (v["person_id"], v["position"], v.get("group"))
        if key in seen:
            continue
        seen.add(key)
        uniq.append(v)

    return uniq

# Cellule 4 : Test avec la nouvelle logique
from lxml import etree

with zipfile.ZipFile(zip_path, "r") as zf:
    # Prend le premier XML
    with zf.open(xml_files[0]) as f:
        tree = etree.parse(f)
        root = tree.getroot()
        
        print(f"📌 Nom de la racine: {root.tag.split('}')[-1]}")
        
        # Test extraction
        actors = root.xpath(".//*[local-name()='acteurRef']")
        print(f"👥 acteurRef trouvés à la racine: {len(actors)}")
        
        # Test extraction votes avec la fonction
        votes = _extract_votes(root)
        print(f"\n📊 RÉSULTAT FINAL:")
        print(f"   Votes extraits: {len(votes)}")
        if votes:
            print(f"   Exemple vote 1: {votes[0]}")
            if len(votes) > 1:
                print(f"   Exemple vote 2: {votes[1]}")

✅ ZIP déjà en cache
📁 Fichiers XML trouvés dans le ZIP: 5294
  - xml/VTANR5L17V3252.xml
  - xml/VTANR5L17V1392.xml
  - xml/VTANR5L17V3913.xml
  - xml/VTANR5L17V3096.xml
  - xml/VTANR5L17V836.xml
📌 Nom de la racine: scrutin
👥 acteurRef trouvés à la racine: 292
🔍 Nombre d'acteurRef trouvés: 292
✅ Votes extraits: 292

📊 RÉSULTAT FINAL:
   Votes extraits: 292
   Exemple vote 1: {'person_id': 'PA793432', 'position': 'AGAINST', 'group': 'PO845401', 'constituency': None, 'name': None}
   Exemple vote 2: {'person_id': 'PA841495', 'position': 'AGAINST', 'group': 'PO845401', 'constituency': None, 'name': None}
